In [5]:
import tensorflow as tf
import time

%run utils.ipynb

In [2]:
batch_size = 40
total_layers = 10
internal_dimension = 400
linear_layer_dimension = 30
#relu_layers = 2
output_dimension = 2

In [7]:
#testing
#get hits dataframe
hits, cells, particles, truth = load_single_train_event(1050)

#transform to cylindrical
hits_trans = trans_to_cylindrical (hits)

hits_array = np.array (hits_trans.values)

r_values = []
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 2)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 4)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 6)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 8)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 13) & (hits_trans['layer_id'] == 2)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 13) & (hits_trans['layer_id'] == 4)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 13) & (hits_trans['layer_id'] == 6)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 13) & (hits_trans['layer_id'] == 8)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 17) & (hits_trans['layer_id'] == 2)])['r'].get_values()))
r_values.append (np.average ((hits_trans.loc[(hits_trans['volume_id'] == 17) & (hits_trans['layer_id'] == 4)])['r'].get_values()))
print (r_values)

[32.31738261840044, 72.15091393575189, 116.0946312221527, 172.07253929769357, 260.3569293324722, 360.3101542051449, 500.2151479081191, 660.12508634391, 820.1546870692688, 1020.1106909393304]


In [22]:
class predict_engine:
    def create_layer_learner (input_layers, x):
        input_dimension = 2*(input_layers)
        x_trimmed = x[:,:input_dimension]
        #x_trimmed = tf.slice (x, [0,0], [x.shape[0],input_dimension])
        W = tf.Variable (tf.random_normal ([input_dimension,internal_dimension], stddev = 1/np.sqrt (internal_dimension)))
        b = tf.Variable (tf.zeros ([internal_dimension]))
        y = tf.nn.leaky_relu (tf.add (tf.matmul (x_trimmed, W), b))
        W = tf.Variable (tf.random_normal ([internal_dimension,linear_layer_dimension], stddev = 1/np.sqrt (internal_dimension)))
        b = tf.Variable (tf.zeros ([linear_layer_dimension]))
        y = tf.nn.leaky_relu (tf.add (tf.matmul (y, W), b))
        W = tf.Variable (tf.random_normal ([linear_layer_dimension, output_dimension], stddev = 1/np.sqrt (linear_layer_dimension)))
        b = tf.Variable (tf.zeros ([output_dimension]))
        y = tf.add (tf.matmul (y, W), b)
        return y
    
    def __init__ (self):
        self.x = tf.placeholder (tf.float32, [None, 20])
        y_target = self.x[:,6:20]
        y = []
        for i in range (3,10):
            y.append (predict_engine.create_layer_learner (i, self.x))
        self.y = tf.concat (y,1)
        self.mean_squared_error = tf.reduce_mean (tf.square (tf.subtract (self.y, y_target)))
        optimizer = tf.train.AdamOptimizer (learning_rate = 0.001)
        self.minimize_op = optimizer.minimize (self.mean_squared_error)
        self.session = tf.Session()
        self.session.run (tf.global_variables_initializer())
        
    def train(self):
        input_layers = total_layers-1
        counter = 0
        mseList = []

        for batch in next_batch (10,batch_size,5, ideal = True, full_data=True):
            batch_array = np.array (batch[0])[:,:,1:3]
            for i in range(len(batch_array)):
                if (np.abs (batch_array[i, 0, 0]) > np.pi/2):
                    do_quadrant_shift = True
                else:
                    do_quadrant_shift = False
                for j in range(len(batch_array[i])):
                    if (do_quadrant_shift):
                        batch_array[i, j, 0] = quadrant_shift (batch_array[i, j, 0])
                    batch_array[i,j,0] = batch_array[i,j,0]/np.pi 
                    batch_array[i,j,1] = batch_array[i,j,1]/1200 
            batch_array_flattened = [batch_array[i].flatten() for i in range(len(batch_array))]
            batch_array_flattened = np.array (batch_array_flattened)    

            print ("loss is ...")
            prediction = self.session.run (self.y, feed_dict = {self.x:batch_array_flattened[0:1]})
            loss = self.session.run (self.mean_squared_error, feed_dict = {self.x:batch_array_flattened})
            print (loss)
            mseList.append (loss)
            self.session.run (self.minimize_op, feed_dict = {self.x:batch_array_flattened})

            predicted = []
            for i in range(0, total_layers-3):
                predicted.append (prediction[0,2*i])

            plt.scatter (batch_array[0][:,0], range (len(batch_array[0])), color = 'blue')
            plt.scatter (predicted, range (3, input_layers+1), color = 'cyan')
            plt.axis ([-1,2,-1,input_layers+1])
            plt.show ()
            plt.scatter (range(len(mseList)), np.log(mseList))
            plt.show()
            print (counter)

            counter += 1
            if (counter >= 1000):
                break
                
    def load(self):
        saver = tf.train.Saver()
        saver.restore (self.session, './checkpoints/allv1/model.ckpt')

    def pred(self, inputs):#inputs are a list of [r, phi, z, l, v] with length 10
        inputs = np.array (inputs)
        for i in range (len (inputs)):
            inputs[i,1] = inputs[i,1]/np.pi
            inputs[i,2] = inputs[i,2]/1200
        trimmed_inputs = [inputs[:,1:3].flatten()]
        padding = [np.zeros (20 - len (trimmed_inputs[0]))]
        trimmed_inputs = np.concatenate ((trimmed_inputs, padding), axis = 1)
        prediction = self.session.run (self.y, feed_dict = {self.x:trimmed_inputs})
        toReturn = np.zeros ([10,3])
        for i in range (2,9):
            toReturn[i,0] = r_values[i+1]
            toReturn[i,1] = prediction[0,2*i-4]*np.pi
            toReturn[i,2] = prediction[0,2*i-3]*1200
        return toReturn
        #should return 10x3.   track x [r, phi, z]

    def close(self):
        session.close()